In [1]:
import os, sys, warnings, re
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from argparse import Namespace

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import MinMaxScaler, StandardScaler

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Make sure we can import your project modules
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from ml.models.seq2seq_lstm import Seq2SeqLSTM
from ml.models.transformer import TimeSeriesTransformer

In [2]:
# -----------------------------
# Config
# -----------------------------
DATA_PATH = "../dataset/combined_with_cluster_feature.csv"   # clustered dataset
DEVICE    = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Choose station + model here
STATION   = "ElBorn"     # one of: "ElBorn", "LesCorts", "PobleSec"
MODEL_SEL = "Seq2Seq"    # "Seq2Seq" or "Transformer"

L = 10  # num_lags
H = 6   # forecast steps
TARGETS = ['rnti_count','rb_down','rb_up','down','up']  # T=5

CKPT_SEQ2SEQ = "seq2seq_cluster_huber.pt"
CKPT_TRANS   = "transformer_multistep_cluster.pt"

In [3]:
# -----------------------------
# Helpers
# -----------------------------
def clamp_nonneg(x):
    return np.maximum(x, 0.0)

def ensure_time(df):
    """Ensure a proper datetime 'time' column exists and sorted."""
    if 'time' not in df.columns:
        raise ValueError("Dataset must have a 'time' column.")
    df = df.copy()
    df['time'] = pd.to_datetime(df['time'], errors='coerce')
    df = df.dropna(subset=['time']).sort_values('time')
    return df

def split_xy_build_single_sample(df_station, L, H, targets, x_scaler_type='minmax', y_scaler_type='minmax'):
    """
    From a single-station dataframe, build a single sample at the last valid index:
      - build lag features for ALL numeric X columns
      - build multi-step targets for 'targets'
      - return: (X_sample_scaled [1,L,D], y_future_scaled [1,H,T], y_scaler, lag_times, future_times)
      - also return lagged ORIGINAL target values for plotting (inverse from df directly)
    """
    df = df_station.copy()
    df = ensure_time(df)

    # Separate inputs and targets
    X_raw = df.drop(columns=[c for c in targets if c in df.columns], errors='ignore')
    # Keep only numeric for X
    X_raw = X_raw.select_dtypes(include=[np.number]).copy()

    # Targets
    y_raw = df[targets].copy()

    # Fit scalers (minmax by default, to mimic your training)
    x_scaler = MinMaxScaler() if x_scaler_type == 'minmax' else StandardScaler()
    y_scaler = MinMaxScaler() if y_scaler_type == 'minmax' else StandardScaler()

    X_scaled = pd.DataFrame(x_scaler.fit_transform(X_raw), columns=X_raw.columns, index=X_raw.index)
    y_scaled = pd.DataFrame(y_scaler.fit_transform(y_raw), columns=y_raw.columns, index=y_raw.index)

    # Generate lag features for X
    X_lagged = []
    lag_cols = []
    for col in X_scaled.columns:
        for lag in range(1, L+1):
            X_lagged.append(X_scaled[col].shift(lag))
            lag_cols.append(f"{col}_lag_{lag}")
    X_lagged = pd.concat(X_lagged, axis=1)
    X_lagged.columns = lag_cols

    # Build multi-step future targets on scaled space
    y_future_cols = []
    for step in range(1, H+1):
        for tcol in targets:
            y_future_cols.append(y_scaled[tcol].shift(-step))
    y_future = pd.concat(y_future_cols, axis=1)
    y_future.columns = [f"{c}_step_{i}" for i in range(1, H+1) for c in targets]

    data = pd.concat([df[['time']], X_lagged, y_future, y_scaled], axis=1).dropna()
    # We'll select the last fully valid row as our anchor
    last_idx = data.index[-1]

    # Reconstruct X window [L, D] from columns
    X_final = data.loc[:, X_lagged.columns].to_numpy()    # [N, L*D_all]
    N, LD   = X_final.shape
    D_all   = LD // L
    X_final = X_final.reshape(N, L, D_all)

    # reshape y_future to [N,H,T]
    y_final = data.loc[:, y_future.columns].to_numpy().reshape(N, H, len(targets))

    # select the last sample
    X_sample = X_final[-1:, :, :]      # [1,L,D]
    y_sample = y_final[-1:, :, :]      # [1,H,T]  (scaled)

    # Times to plot
    # lag times: last L time stamps ending at last_idx's time; compute based on original df time
    # We need the row position in df
    pos_in_df = df.index.get_loc(last_idx)
    if isinstance(pos_in_df, slice):
        # get the end of slice
        pos_in_df = pos_in_df.stop - 1
    # take last L times (ensure bounds)
    start_pos = max(0, pos_in_df - L)
    lag_times = df.iloc[start_pos:pos_in_df]['time'].tolist() + [df.iloc[pos_in_df]['time']]
    lag_times = lag_times[-L:]  # exactly L
    # future times: next H rows if they exist, else infer by frequency (use df time gaps)
    future_times = []
    if pos_in_df + H < len(df):
        future_times = df.iloc[pos_in_df+1:pos_in_df+1+H]['time'].tolist()
    else:
        # infer frequency from median delta
        times = df['time'].sort_values().values
        if len(times) >= 2:
            freq = pd.to_timedelta(pd.Series(times).diff().dropna()).median()
        else:
            freq = pd.to_timedelta("2min")
        t0 = df.iloc[pos_in_df]['time']
        future_times = [t0 + (i+1)*freq for i in range(H)]

    # Also capture the ORIGINAL (unscaled) lag target values to plot left side
    lag_targets_orig = df[targets].iloc[pos_in_df-L+1:pos_in_df+1].to_numpy()  # [L,T]

    return X_sample, y_sample, y_scaler, lag_times, future_times, lag_targets_orig

def load_model(model_sel, D, T, H, device):
    if model_sel == "Seq2Seq":
        model = Seq2SeqLSTM(
            input_size=D, hidden_size=64,
            output_size=T, forecast_steps=H, num_layers=1
        ).to(device)
        model.load_state_dict(torch.load(CKPT_SEQ2SEQ, map_location=device), strict=True)
    elif model_sel == "Transformer":
        model = TimeSeriesTransformer(
            input_size=D, output_size=T, forecast_steps=H,
            d_model=128, nhead=4,
            num_encoder_layers=2, num_decoder_layers=2,
            dim_feedforward=256, dropout=0.1
        ).to(device)
        model.load_state_dict(torch.load(CKPT_TRANS, map_location=device), strict=True)
    else:
        raise ValueError("MODEL_SEL must be 'Seq2Seq' or 'Transformer'")
    model.eval()
    return model

def inverse_targets_scaled(y_scaled_2d, y_scaler):
    """
    Inverse-transform a 2D array [N, T] (here we pass [H, T] or [1, T]).
    """
    return y_scaler.inverse_transform(y_scaled_2d)

In [4]:
# -----------------------------
# 1) Load & filter data
# -----------------------------
df_all = pd.read_csv(DATA_PATH)
if 'District' not in df_all.columns:
    raise ValueError("Expected a 'District' column in the dataset.")

df_station = df_all[df_all['District'] == STATION].copy()
if df_station.empty:
    raise ValueError(f"No rows for station '{STATION}' found in {DATA_PATH}.")
df_station = ensure_time(df_station)

In [5]:
# -----------------------------
# 2) Build a single sample & times
# -----------------------------
X_sample, y_sample_scaled, y_scaler, lag_times, future_times, lag_targets_orig = \
    split_xy_build_single_sample(df_station, L=L, H=H, targets=TARGETS,
                                 x_scaler_type='minmax', y_scaler_type='minmax')

In [6]:
# -----------------------------
# 3) Load model & predict
# -----------------------------
_, L_, D = X_sample.shape
T = len(TARGETS)

model = load_model(MODEL_SEL, D=D, T=T, H=H, device=DEVICE)

with torch.no_grad():
    xb = torch.tensor(X_sample, dtype=torch.float32, device=DEVICE)   # [1,L,D]
    if MODEL_SEL == "Seq2Seq":
        preds_scaled = model(xb, teacher_forcing_ratio=0.0)           # [1,H,T]
    else:
        preds_scaled = model(xb)                                      # [1,H,T]
preds_scaled = preds_scaled.squeeze(0).cpu().numpy()                  # [H,T]

# Inverse-transform predictions to ORIGINAL space
preds_orig = inverse_targets_scaled(preds_scaled, y_scaler)           # [H,T]
preds_orig = clamp_nonneg(preds_orig)

In [7]:
# -----------------------------
# 4) Plot 5 line charts (Plotly)
# -----------------------------
fig = make_subplots(rows=5, cols=1, shared_xaxes=False,
                    vertical_spacing=0.07,
                    subplot_titles=TARGETS)

for i, tgt in enumerate(TARGETS, start=1):
    # Left series (lagged ground truth, ORIGINAL)
    y_lag = lag_targets_orig[:, TARGETS.index(tgt)]   # [L]
    x_lag = lag_times

    # Right series (predictions t+1..t+6)
    y_pred = preds_orig[:, TARGETS.index(tgt)]        # [H]
    x_pred = future_times

    # Plot lagged segment as a line
    fig.add_trace(
        go.Scatter(x=x_lag, y=y_lag, mode='lines+markers',
                   name=f"{tgt} (lags)", showlegend=(i==1)),
        row=i, col=1
    )

    # Plot predicted segment as a line with a different dash
    fig.add_trace(
        go.Scatter(x=x_pred, y=y_pred, mode='lines+markers',
                   name=f"{tgt} (pred t+1..t+6)", line=dict(dash='dash'), showlegend=(i==1)),
        row=i, col=1
    )

    # Add vertical separator at the last lag time
    sep_time = x_lag[-1]
    fig.add_vline(x=sep_time, line_width=2, line_dash="dot", line_color="gray", row=i, col=1)

    # Aesthetic touches
    fig.update_yaxes(title_text=tgt, row=i, col=1)

fig.update_layout(
    title=f"{STATION} — {MODEL_SEL} forecasts (lags L={L} → t+1..t+{H})",
    height=300*5, width=1100,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1.0),
    hovermode="x unified",
)

fig.show()